In [2]:
import gym
import random
from keras import Sequential
from collections import deque
from keras.layers import Dense
from keras.optimizers import adam
import matplotlib.pyplot as plt
from keras.activations import relu, linear

import numpy as np

Using TensorFlow backend.


In [10]:
env = gym.make('LunarLander-v2')
# Landing pad is always at coordinates (0,0). 
# Coordinates are the first two numbers in state vector. 
# Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. 
# If lander moves away from landing pad it loses reward back. 
# Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. 
# Each leg ground contact is +10. 
# Firing main engine is -0.3 points each frame. Solved is 200 points. 
# Landing outside landing pad is possible. 
# Fuel is infinite, so an agent can learn to fly and then land on its first attempt. 
# Four discrete actions available: do nothing, fire left orientation engine, fire main engine,
#     fire right orientation engine.

print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)
obs_size = env.observation_space.shape[0]
print(obs_size)

Box(8,)
[inf inf inf inf inf inf inf inf]
[-inf -inf -inf -inf -inf -inf -inf -inf]
8


In [9]:
test_games_num = 100
test_scores = []
max_steps_in_game = 1000
for game in range(test_games_num):
    score = 0
    observation = env.reset()  # get initial state
    for step in range(max_steps_in_game):
        env.render()  # show gif
#         print(observation)  # print state vector 8, fixed for environment
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)  # step returns 4 parameters
        score +=reward
        if done:  # game over need reset
#             print("Episode finished after {} timesteps".format(step+1))
            print(score)
            break
    test_scores.append(score)

-76.09117154009367
-131.60169391064753
-408.2377808717626
-182.96031703311473
-359.96201733608706
-121.26160167794612
-216.90136446776316
-501.2584007026191
-74.72028169081048
-127.00197073999618
-109.30880091204284
-178.78587977693718
-115.91864816550228
-197.01982445381321
-98.54451388120903
10.941737404175583
-289.84081703933884
-321.60963417668245
-127.96548625071343
-283.4306951108398
-105.84525848540619
-90.68991666245802
-197.66198856399382
-104.29179822725567
-333.58161480855966
-388.3357336889067
-147.20942529575166
-118.11830075591402
-123.33705015373206
-70.02705600225505
4.838136517284781
-106.53878603350991
-152.88092331188983
-368.18264031134527
-173.95224217521542
-68.71983727579436
-227.9824360103706
-297.1124843577488
-89.53212830988443
-100.624475821276
-75.33534800608732
-71.3494698211212
-232.4138201448486
-79.53578106721253
-84.90792141697231
-180.88894967853898
-52.64260320556362
-221.9220148920389
-93.9058169619801
-93.29161706209143
-229.02902394468742
-341.4376

In [11]:
print("Average reward on test 100 games: ", np.mean(test_scores))

Average reward on test 100 games:  -172.83697185238046


In [12]:
test_scores = np.array(test_scores)
print("Number of games in 100 where reward")
print("less than 0: ", len(test_scores[test_scores<0]))
print("equals 0: ", len(test_scores[test_scores==0]))
print("more than 0: ", len(test_scores[test_scores>0]))
print("equals or more than 100: ", len(test_scores[test_scores>=100]))
print("equals or more than 200: ", len(test_scores[test_scores>=200]))

Number of games in 100 where reward
less than 0:  97
equals 0:  0
more than 0:  3
equals or more than 100:  0
equals or more than 200:  0
